# Defect Prediction on production lines by VALEO
# Challenge context
Valeo is a French global automotive supplier headquartered in France, listed on the Paris Stock Exchange (CAC-40 Index). It supplies a wide range of products to automakers and the aftermarket. The Group employs 113,600 people in 33 countries worldwide. It has 186 production plants, 59 R&D centers and 15 distribution platforms. Its strategy is focused on innovation and development in high-growth potential regions and emerging countries. Valeo ranked as France's leading patent filer from 2016 to 2018.
 
# Challenge goals
The goal of the challenge is to predict defect on starter motor production lines. During production samples assembly, different values (torques, angles ...) are measured on different mounting stations. At the end of the line, additional measures are performed on two test benches in order to isolate defects. As a result, samples are tagged ‘OK’, ‘KO’. We would like to design a model that could identify such defects before the test bench step.

# Data description
**ID** = PROC_TRACEINFO = it’s a unique code given to the product. Example : I-B-XA1207672-190701-00494.

XA1207672 is the reference.
190701 is the date: here 01st of July of year 2019.
00494 is the unique code given to the product, whatever it happens, the product will have this id number frozen forever.
This number is increased by 1 each time we process a new product, every 12s. So for example : I-B-XA1207672-190701-00495 is the next product.

**Inputs** : Input features are measures collected on different assembly stations with the sensors or devices connected to Programmable Logic Controllers which are storing all of them to keep the full quality traceability. (Examples : OP070_V_1_angle_value, OP120_Rodage_I_value, etc…).

**Output** : This is the result value of OP130 (test bench). Value 0 is assigned to OK samples (passed) and value 1 is assigned to KO samples (failed). This is the combined result of multiple electrical, acoustic and vibro-acoustic tests.

**The target** is to find the best prediction : Output = f (inputs). The dataset contains 34515 training samples and 8001 test samples.

# Benchmark description
We expect a AUROC more than 0.675 which can easily be obtained with a basic **Naive Bayes classifier**. This AUROC value had been obtained using some techniques for unbalanced classes handling and not only with the challenge metric. That's the main reason why our AUROC value of 0.675 is different from the benchmark value (0.5904).

***

# Tested Models score:
* Decision Tree: 
* Random Forest: 

Oversampling

    Nan Filled with KNN

Undersampling



ideas:

* outliers filtering isoltion forest
* filling nan values with knn
* oversampling minority class: smote
* Date time featurer engineering

* ensemble method: multiple models that test the statistical deviance of each of the H0 rejected feature, predict "KO" if even of of them are significantly different.


# 2. Data Pre-processing

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
nb_path = os.path.abspath(".")

In [2]:
nb_path = os.path.abspath(".")
InteractiveShell.ast_node_interactivity = "all"
sns.set(color_codes=True)

In [3]:
X_validation = pd.read_csv(os.path.join(nb_path,"testinputs.csv")) 
dataset = pd.read_csv(os.path.join(nb_path,"traininginputs.csv")).merge(pd.read_csv(os.path.join(nb_path,"trainingoutput.csv")), on="PROC_TRACEINFO")

In [4]:
target = "Binar OP130_Resultat_Global_v"
code = "PROC_TRACEINFO"
capuchon_measure = "OP100_Capuchon_insertion_mesure"
selected_measures = ['OP070_V_1_angle_value','OP090_SnapRingPeakForce_value', 'OP090_SnapRingFinalStroke_value','OP070_V_1_torque_value','OP090_SnapRingMidPointForce_val', 'OP070_V_2_torque_value']

In [464]:
dataset

,PROC_TRACEINFO,OP070_V_1_angle_value,OP090_SnapRingPeakForce_value,OP070_V_2_angle_value,OP120_Rodage_I_mesure_value,OP090_SnapRingFinalStroke_value,OP110_Vissage_M8_torque_value,OP100_Capuchon_insertion_mesure,OP120_Rodage_U_mesure_value,OP070_V_1_torque_value,OP090_StartLinePeakForce_value,OP110_Vissage_M8_angle_value,OP090_SnapRingMidPointForce_val,OP070_V_2_torque_value,Binar OP130_Resultat_Global_v
0,I-B-XA1207672-190429-00688,180.4,190.51,173.1,113.64,12.04,12.16,NaN,11.97,6.62,26.37,18.8,109.62,6.60,0
1,I-B-XA1207672-190828-00973,138.7,147.70,163.5,109.77,12.12,12.19,0.39,11.97,6.41,21.03,18.5,105.48,6.40,0
2,I-B-XA1207672-190712-03462,180.9,150.87,181.2,109.79,11.86,12.24,NaN,11.97,6.62,25.81,17.5,100.03,6.61,0
3,I-B-XA1207672-190803-00051,173.5,159.56,151.8,113.25,11.82,12.35,0.39,11.97,6.62,24.62,15.6,104.94,6.61,0
4,I-B-XA1207672-190508-03248,174.5,172.29,177.5,112.88,12.07,12.19,NaN,11.97,6.62,29.22,33.6,99.19,6.61,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34510,I-B-XA1207672-190520-00031,158.2,168.40,180.6,110.65,11.88,12.23,NaN,11.98,6.61,24.03,14.8,98.76,6.61,0
34511,I-B-XA1207672-190906-00270,139.4,146.24,154.5,108.14,12.04,12.27,0.41,11.97,6.41,21.52,15.5,98.37,6.40,0
34512,I-B-XA1207672-190429-00930,165.5,157.34,149.8,112.77,12.09,12.17,NaN,11.97,6.62,22.65,24.4,92.92,6.62,0
34513,I-B-XA1207672-190424-02120,152.4,165.07,140.0,111.19,12.06,12.34,NaN,11.97,6.60,20.77,13.6,101.21,6.60,0


## Data Splitting

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
trainset, testset = train_test_split(dataset, test_size=0.2, random_state=0)

## Pre-processing Steps

In [7]:
# X_resampled, y_resampled = SMOTE().fit_resample(X, y)
# print(sorted(Counter(y_resampled).items()))

# clf_smote = LinearSVC().fit(X_resampled, y_resampled)
# X_resampled, y_resampled = ADASYN().fit_resample(X, y)
# print(sorted(Counter(y_resampled).items()))

# clf_adasyn = LinearSVC().fit(X_resampled, y_resampled)

In [8]:
def removing_outliers(df):
#     from sklearn.ensemble import isolationforest
    
#     iso = IsolationForest(contamination=0.1)
#     yhat = iso.fit_predict(X_train)
#     mask = yhat != -1
#     X_train, y_train = X_train[mask, :], y_train[mask]
    
    return df

In [9]:
def feature_engineering(df):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-9-e8816d0a069b>, line 2)

In [15]:
def rebalancing(df):
    
#     # undersampling
#     df = df.sample(frac=1, random_state=0) # Lets shuffle the data before creating the subsamples
    
#     ko_df = df[df[target]==1]
#     ok_df = df[df[target]==0][:ko_df.shape[0]]  # amount of KO classes ko_df.shape[0] rows.
    
#     df = pd.concat([ok_df,ko_df], axis=0)
    
    return df

In [16]:
def imputation(df):
    #df["is na"] = df["OP100_Capuchon_insertion_mesure"].isna()
    # outliers handling
    # fill na with smote??? make sense?
    # fill na with knn
#     df = df.dropna(axis=0)
    
    df = df.drop("OP100_Capuchon_insertion_mesure", axis=1)
    return df

In [17]:
def preprocessing(df):
    df = imputation(df)
#     df = rebalancing(df)
    
    X = df.drop([target,code], axis=1)
    y = df[target]
    
#     X = df[selected_measures]
#     y = df[target]

#     X = df[['OP070_V_1_torque_value','OP070_V_2_torque_value']]
#     y = df[target]

    
    print(y.value_counts())
    
    return X, y

In [18]:
X_train, y_train = preprocessing(trainset)
X_test, y_test = preprocessing(testset)

0    27374
1      238
Name: Binar OP130_Resultat_Global_v, dtype: int64
0    6836
1      67
Name: Binar OP130_Resultat_Global_v, dtype: int64


In [19]:
X_train.head()

,OP070_V_1_angle_value,OP090_SnapRingPeakForce_value,OP070_V_2_angle_value,OP120_Rodage_I_mesure_value,OP090_SnapRingFinalStroke_value,OP110_Vissage_M8_torque_value,OP120_Rodage_U_mesure_value,OP070_V_1_torque_value,OP090_StartLinePeakForce_value,OP110_Vissage_M8_angle_value,OP090_SnapRingMidPointForce_val,OP070_V_2_torque_value
20907,146.8,157.47,142.3,115.35,12.09,12.40,11.97,6.41,21.09,12.5,95.90,6.42
26482,146.8,178.43,154.9,117.87,12.08,12.29,11.97,6.62,25.28,14.5,97.66,6.61
22105,156.3,149.65,158.3,109.18,12.12,12.16,11.97,6.41,17.80,16.0,81.39,6.41
25164,183.5,181.87,157.2,115.00,12.07,12.18,11.98,6.60,23.89,18.9,100.32,6.60
991,161.7,164.63,157.4,111.54,12.08,12.24,11.97,6.62,19.67,22.5,105.25,6.61


In [20]:
y_train.head()

20907    0
26482    0
22105    0
25164    0
991      0
Name: Binar OP130_Resultat_Global_v, dtype: int64

# 3. Model Building

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer, MissingIndicator
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE, KMeansSMOTE, SMOTENC, SVMSMOTE
from imblearn.combine import SMOTEENN, SMOTETomek

In [137]:
# best performer: auc=0.6609422897216668
# prepro: drop capuchon
# model = make_pipeline(RandomUnderSampler(random_state=0), SelectKBest(f_classif, k=7), LogisticRegression(random_state=0))

# 2nd performer: auc: 0.6406688034374645
# prepro:drop capuchon
# better training score than LR
# model = make_pipeline(RandomUnderSampler(random_state=0), StandardScaler(), SelectKBest(f_classif, k=10), SVC(random_state=0, kernel='rbf'))


model = make_pipeline(SMOTETomek(random_state=0, n_jobs=-1), StandardScaler(), SelectKBest(f_classif, k=10), SVC(random_state=0, kernel='rbf'))
# model = make_pipeline(RandomUnderSampler(random_state=0), SelectKBest(f_classif, k=9), RandomForestClassifier(random_state=0))
# model = make_pipeline(RandomUnderSampler(random_state=0), PolynomialFeatures(), SelectKBest(f_classif, k='all'), RandomForestClassifier(random_state=0))
# model = make_pipeline(RandomUnderSampler(random_state=0), PolynomialFeatures(), PCA(n_components=10, random_state=0), RandomForestClassifier(random_state=0))
# model = make_pipeline(RandomUnderSampler(random_state=0), StandardScaler(), SelectKBest(f_classif, k=9), XGBClassifier())
# model = make_pipeline(SMOTE(random_state=0, n_jobs=-1), RandomForestClassifier(random_state=0))
# model = DecisionTreeClassifier(random_state=0)
# model = RandomForestClassifier(random_state=0)

In [138]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import learning_curve
from imblearn.metrics import classification_report_imbalanced

In [139]:
def evaluation(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    print(roc_auc_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
#     print(classification_report_imbalanced(y_test, y_pred))
    
    N, train_score, val_score = learning_curve(model, X_train, y_train,
                                          cv=4, scoring='roc_auc',
                                               train_sizes=np.linspace(0.1, 1, 10), n_jobs=-1)
    plt.figure(figsize=(12, 8))
    plt.plot(N, train_score.mean(axis=1), label='train score')
    plt.plot(N, val_score.mean(axis=1), label='validation score')
    plt.legend()
    

In [140]:
import warnings
warnings.filterwarnings('ignore')

evaluation(model)

0.5456822091997588
[[5522 1314]
 [  48   19]]
              precision    recall  f1-score   support

           0       0.99      0.81      0.89      6836
           1       0.01      0.28      0.03        67

    accuracy                           0.80      6903
   macro avg       0.50      0.55      0.46      6903
weighted avg       0.98      0.80      0.88      6903



KeyboardInterrupt: 

In [440]:
from sklearn.metrics import SCORERS
sorted(SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

In [616]:
pd.DataFrame(model.feature_importances_, index=X_train.columns).plot.bar()

AttributeError: 'Pipeline' object has no attribute 'feature_importances_'

## 